In [1]:
import pandas as pd
import os

## Loading data

In [2]:
path = os.getcwd()[0:-5] + '\\Data\\kprm-with-salary-update-20231115.json'

df = pd.read_json(path, lines=True)

# Dropping unnecessary columns:
# '6' has been split on 2 columns due to an error during the download,
# while '8' is a duplicate of '9' with minor differences. However, '9' is
# believed to be the correct version.
df.drop(columns=[6,8], inplace=True)

# Renaming columns.
df.columns = ['Ad ID', 'Job title', 'Employer (entity)', 'Employer location', 'Office address', 'Workplace', 'Department', 'Posted on', 'Expiration date', 'Detailed recruitment results', 'Salary (monthly)', 'Vacancies', 'Full-time ratio', 'Recruitment results', 'Description of duties', 'Required level of education', 'Prerequisites', 'Optional requirements', 'Application instructions', 'Number of views']

In [3]:
print(df['Recruitment results'].nunique())

1


In [4]:
# The data contains no active posts at the time of the download:
# the column "Recruitment results" contains only one unique value - "Koniec naboru"
# (recruitment finished).
# Thus, we can drop the column.
df.drop(columns=["Recruitment results"], inplace=True)

## Cleansing

#### Reducing data size
<span style="font-size:1.05em;">We're removing unnecessary text from rows, such as "NR" (an abbreviation for number) in every cell of the "Ad ID" column.</span>

In [5]:
for index, row in df.iterrows():
    if row['Ad ID'][0] == 'N' and row['Ad ID'][2] == ' ':
        df.at[index, 'Ad ID'] = row['Ad ID'][3:]
    
    # Removing "Adres urzędu: " from each row.
    if row['Office address'][12] == ':' and row['Office address'][13] == ' ':
        df.at[index, 'Office address'] = row['Office address'][14:]
    elif row['Office address'][12] == ':':
        df.at[index, 'Office address'] = row['Office address'][13:]
        
    # Removing "Miejsce wykonywania pracy: " from each row.
    if row['Workplace'][25] == ':' and row['Workplace'][26] == ' ':
        df.at[index, 'Workplace'] = row['Workplace'][27:]
    elif row['Workplace'][25] == ':':
        df.at[index, 'Workplace'] = row['Workplace'][26:]

    # Removing "Wykształcenie: " from each row. 
    if row['Required level of education'][13] == ':' and row['Required level of education'][14] == ' ':
        df.at[index, 'Required level of education'] = row['Required level of education'][15:]
    elif row['Required level of education'][13] == ':':
        df.at[index, 'Required level of education'] = row['Required level of education'][14:]
        
    # Removing "Wyniki naboru: " from each row.
    if row['Detailed recruitment results'][14] == ' ' and row['Detailed recruitment results'][13] == ':':
        df.at[index, 'Detailed recruitment results'] = row['Detailed recruitment results'][15:]
    elif row['Detailed recruitment results'][13] == ':':
        df.at[index, 'Detailed recruitment results'] = row['Detailed recruitment results'][14:]
    
    # Removing "Liczba odwiedzin: " from each row.
    # One additional condition since the first character in most entries is a space.
    if row['Required level of education'][0] == ' ':
        if row['Number of views'][18] == ':' and row['Number of views'][19] == ' ':
            df.at[index, 'Number of views'] = row['Number of views'][20:]
        elif row['Number of views'][18] == ':' and row['Number of views'][1].lower() == 'l':
            df.at[index, 'Number of views'] = row['Number of views'][19:]
    else:
        if row['Number of views'][17] == ':' and row['Number of views'][18] == ' ':
            df.at[index, 'Number of views'] = row['Number of views'][19:]
        elif row['Number of views'][16] == ':' and row['Number of views'][0].lower() == 'l':
            df.at[index, 'Number of views'] = row['Number of views'][17:]

        
df.head()

,Ad ID,Job title,Employer (entity),Employer location,Office address,Workplace,Department,Posted on,Expiration date,Detailed recruitment results,Salary (monthly),Vacancies,Full-time ratio,Description of duties,Required level of education,Prerequisites,Optional requirements,Application instructions,Number of views
0,131450,główny specjalista,Ministerstwo Rolnictwa i Rozwoju Wsi w Warszawie,Warszawa,ul. Wspólna 30 00-930 Warszawa,Warszawa,w Wydziale Obsługi Płacowej w Biurze Finansowym,15 grudnia 2023,29 grudnia 2023,anulowano nabór|,"7009,44 zł brutto",1,1,[nalicza wynagrodzenia pracowników oraz zasiłk...,wyższe,[3 lata stażu pracy w obszarze wynagrodzeń i u...,[Wykształcenie wyższe ekonomiczne lub ekonomic...,"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",91
1,131558,księgowy,Powiatowy Inspektorat Weterynarii w Piszu,Pisz,ul. Warszawska 38 12-200 Pisz,Pisz Powiatowy Inspektorat Weterynarii ul. War...,w zespole ds. finansowo-księgowych i administr...,15 grudnia 2023,29 grudnia 2023,anulowano nabór|,nie podano wynagrodzenia,1,1,[Wykonuje zadania księgowego w jednostce budże...,wyższe ekonomiczne,"[doświadczenie zawodowe: co najmniej 1 rok , W...",[komunikatywność],"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",25
2,131502,starszy specjalista,Główny Urząd Nadzoru Budowlanego w Warszawie,Warszawa,Krucza 38/42 00-926 Warszawa,Warszawa ul. Krucza 38/42,w Biurze Organizacyjnym,14 grudnia 2023,29 grudnia 2023,anulowano nabór|,nie podano wynagrodzenia,1,1,[organizuje proces przeprowadzania naborów do ...,wyższe,"[doświadczenie zawodowe: 1,5 roku w pracy zwią...",[Wykształcenie: wyższe prawnicze lub administr...,"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",34
3,131510,inspektor,Powiatowy Inspektorat Nadzoru Budowlanego w Hr...,Hrubieszów,ul. Plac Wolności 13 22-500 Hrubieszów,"Hrubieszów 22-500 Hrubieszów, ul. Plac Wolnośc...",Referat inspekcji i kontroli,13 grudnia 2023,29 grudnia 2023,anulowano nabór|,nie podano wynagrodzenia,1,1,[Prowadzi postępowania administracyjne wynikaj...,średnie budowlane lub architektoniczne,[staż pracy: w administracji publicznej lub w ...,[Wykształcenie: wyższe Wykształcenie: wyższe b...,"[Dokumenty należy złożyć do: 29.12.2023, Decyd...",58
4,131408,główny specjalista,Ministerstwo Rozwoju i Technologii w Warszawie,Warszawa,pl. Trzech Krzyży 3/5 00-507 Warszawa,Warszawa,w Wydziale Wynagrodzeń w Departamencie Budżetu...,13 grudnia 2023,27 grudnia 2023,anulowano nabór|,"od 7009,00 zł do 7447,00 zł brutto",1,1,[prowadzi obsługę płacową dla pracowników zatr...,wyższe ekonomiczne lub inne nieprofilowane wra...,[Doświadczenie zawodowe: minimum 3 lata w obsz...,"[CV i list motywacyjny, Kopie dokumentów potwi...","[Dokumenty należy złożyć do: 27.12.2023, Decyd...",395


#### Making data readable for computers

##### The salary column

<span style="font-size:1.05em;">First, we're checking whether all salaries are given as gross (brutto) or if there are some given as net (netto).</span>

In [6]:
df_netto = df[df['Salary (monthly)'].str.contains('netto')][['Salary (monthly)']]
count_netto = len(df_netto)
print(count_netto)

117


<span style="font-size:1.05em;">Exploring the frequency of different recruitment results.</span>

In [7]:
# Announcements that ended with the employment of a candidate.
success_df = df[df['Detailed recruitment results'].str.contains(r'wyborem kandydatki/kandydata')][['Detailed recruitment results']]
success_count = len(success_df)

In [8]:
# Announcements that ended without employing a candidate.
failure_df = df[df['Detailed recruitment results'].str.contains(r'bez wyboru kandydatki/kandydata')][['Detailed recruitment results']]
failure_count = len(failure_df)

In [9]:
# Announcements that have been cancelled as a results of an error; they are omitted in our analysis.
error1_df = df[df['Detailed recruitment results'].str.contains(r'błędu|błąd')][['Detailed recruitment results']]
error1_count = len(error1_df)

# 
error2_df = df.loc[df['Detailed recruitment results'] == 'anulowano nabór|']
error2_count = len(error2_df)

error_count = error1_count + error2_count
print(f"Ads that led to employment: {success_count}\n"
      f"Ads that did not led to employment: {failure_count}\n"
      f"Ads cancelled as a result of an error: {error1_count}")

Ads that led to employment: 45150
Ads that did not led to employment: 28946
Ads cancelled as a result of an error: 29


In [10]:
indexes_to_remove = list(error1_df.index) + list(error2_df.index) + list(success_df.index) + list(failure_df.index)

In [11]:
df_copy = df
df_copy.drop(indexes_to_remove, axis=0, inplace=True)
df_copy[0:300]

,Ad ID,Job title,Employer (entity),Employer location,Office address,Workplace,Department,Posted on,Expiration date,Detailed recruitment results,Salary (monthly),Vacancies,Full-time ratio,Description of duties,Required level of education,Prerequisites,Optional requirements,Application instructions,Number of views
6,131459,laborant,Główny Inspektorat Ochrony Roślin i Nasiennict...,Rzeszów,Al. Jana Pawła II 11 00-828 Warszawa,"Rzeszów OCL GIORiN w Rzeszowie, ul. Langiewicz...",w Oddziale Centralnego Laboratorium w Rzeszowie,13 grudnia 2023,22 grudnia 2023,anulowano nabórN|abór wprowadzono dwukrotnie w...,nie podano wynagrodzenia,1,1,[przygotowuje próbki do badań laboratoryjnych;...,"średnie biologiczne, rolnicze, bądź pokrewne;",[„W służbie cywilnej nie może być zatrudniona ...,"[Wykształcenie: wyższe biologiczne, rolnicze l...","[Dokumenty należy złożyć do: 22.12.2023, Decyd...",207
10,131315,kontroler weterynaryjny,Powiatowy Inspektorat Weterynarii w Zgierzu,Zgierz,ul. A. Struga 23 95-100 Zgierz,Zgierz Zgierz ul. A.Struga 23,"Zespół ds. bezpieczeństwa żywności, pasz i uty...",08 grudnia 2023,25 grudnia 2023,"anulowano nabórB|rak informacji dla kandydata,...",nie podano wynagrodzenia,1,1,[Nadzór weterynaryjny nad podmiotami produkują...,"średnie zootechnika, technik weterynarii",[Znajomość przepisów weterynaryjnych z zakresu...,[Wykształcenie: wyższe weterynaryjne ],"[Dokumenty należy złożyć do: 25.12.2023, Decyd...",70
13,131159,inspektor weterynaryjny,Powiatowy Inspektorat Weterynarii w Lęborku,Lębork,Ul. Weterynaryjna 1 84-300 Lębork,Lębork ul. Weterynaryjna 1 84-300 Lębork,"w zespole ds. bezpieczeństwa żywności, pasz i ...",06 grudnia 2023,20 grudnia 2023,anulowano nabórB|łędy w ogłoszeniu,nie podano wynagrodzenia,1,1,[wykonuje zdania Inspekcji Weterynaryjnej okre...,wyższe weterynaryjne,"[Prawo wykonywania zawodu lekarza weterynarii,...","[staż pracy: , znajomość języka angielskiego n...","[Dokumenty należy złożyć do: 20.12.2023, Decyd...",67
16,130982,pełnomocnik do spraw ochrony informacji niejaw...,Generalna Dyrekcja Ochrony Środowiska w Warszawie,Warszawa,Al. Jerozolimskie 136 02-305 Warszawa,Warszawa Al. Jerozolimskie 136 (Eurocentrum Of...,Stanowisko do spraw Ochrony Informacji Niejawnych,08 grudnia 2023,20 grudnia 2023,anulowano nabórZ|miana wynika ze zmian organiz...,"3723,00 zł brutto",1,1/2,[zapewnia przestrzeganie przepisów o ochronie ...,wyższe,[doświadczenie zawodowe: co najmniej 1 rok w o...,[doświadczenie zawodowe: co najmniej 1 rok w o...,"[Dokumenty należy złożyć do: 20.12.2023, Decyd...",391
18,131087,starszy inspektor,Opolski Urząd Wojewódzki w Opolu,Opole,ul. Piastowska 14 45-082 Opole,Opole ul. Piastowska 14 45-082 Opole,w Wydziale Prawnym i Nadzoru / Oddział Organiz...,14 grudnia 2023,12 grudnia 2023,anulowano nabórA|nulowanie naboru z uwagi na b...,nie podano wynagrodzenia,1,1,"[Rozpatruje skargi, wnioski i petycje w celu z...",średnie,"[doświadczenie zawodowe: co najmniej 1 rok , Z...",[doświadczenie zawodowe: co najmniej 8 miesięc...,"[Dokumenty należy złożyć do: 12.12.2023, Decyd...",438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5024,124476,inspektor weterynaryjny,Powiatowy Inspektorat Weterynarii w Rawie Mazo...,Rawa Mazowiecka,Mszczonowska 10 96-200 Rawa Mazowiecka,Rawa Mazowiecka,"w zespole ds. bezpieczeństwa żywności, pasz i ...",25 lipca 2023,25 sierpnia 2023,anulowano nabórN|IEKOMPLETNE INFORMACJE,nie podano wynagrodzenia,1,1,[sprawuje nadzór weterynaryjny nad podmiotami ...,wyższe weterynaryjne,[prawo wykonywania zawodu lekarza weterynarii;...,[staż pracy: co najmniej 6 miesięcy ],"[Dokumenty należy złożyć do: 25.08.2023, Decyd...",593
5027,124481,naczelnik wydziału,Ministerstwo Finansów w Warszawie,Warszawa,ul. Świętokrzyska 12 00-916 Warszawa,Warszawa,w Wydziale Procesów Zakupowych i Rejestru Umów...,28 sierpnia 2023,4 sierpnia 2023,nie zatrudniono kandydatki/kandydata,nie podano wynagrodzenia,1,1,"[Kieruje pracą wydziału, w szczególności poprz...",wyż

In [12]:
# ^Wychodzi na to że komunikaty odnośnie efektu ogłoszenia nie są jednolite. Algorytm do korekty.

In [13]:
import re
with open('Data cleansing.ipynb', 'r') as file:
    # Read original notebook file to string
    jupyter = file.read()
    
    # Run a regex based search and replace, wipe all empty outputs properties.
    outputs_tag_removed = re.sub(',\n\s+\"outputs\":\ \[\]', '', jupyter)
    
    # Overwrite original notebook file
    print(outputs_tag_removed, file=open('Data cleansing.ipynb', 'w'))

In [14]:
# Solution from https://github.com/jupyter/nbconvert/issues/1872https://github.com/jupyter/nbconvert/issues/1872